In [31]:
import os
import os.path as op
import numpy as np
import mne
from mne.preprocessing import ICA, create_eog_epochs
from mne import io, combine_evoked
from mne.minimum_norm import make_inverse_operator, apply_inverse
from mne.minimum_norm import read_inverse_operator, make_inverse_operator,source_band_induced_power
mne.set_log_level('warning')
#mne.viz.set_3d_backend('pyvista')
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
from mne import set_config
import pandas as pd
from scipy import stats

In [32]:
# This code sets an environment variable called SUBJECTS_DIR
os.environ['SUBJECTS_DIR'] = '/net/server/data/Archive/prob_learn/freesurfer'
subjects_dir = '/net/server/data/Archive/prob_learn/freesurfer'

In [33]:
# Испытуемый выбран рандомно, просто в качестве примера 
subject='P001'

In [34]:
# the raw file containing the channel location + types
raw_fname = '/net/server/mnt/Archive/prob_learn/experiment/P001/161228/ORIGINAL_TSSS/P001_run2_raw_tsss_mc.fif'
raw = mne.io.read_raw_fif(raw_fname, preload=True) 

<ipython-input-34-fda92aeca6ac>:3: RuntimeWarning: This filename (/net/server/mnt/Archive/prob_learn/experiment/P001/161228/ORIGINAL_TSSS/P001_run2_raw_tsss_mc.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw = mne.io.read_raw_fif(raw_fname, preload=True)


In [35]:
# Берем в качестве примера norisk, fb current = negative
events = np.loadtxt('/net/server/data/Archive/prob_learn/vtretyakova/Nikita_mio_cleaned/events_by_cond_mio_corrected/P001_run2_norisk_fb_cur_negative.txt', dtype='int')

In [36]:
period_start = -1.750
period_end = 2.750

# by default tmin=- 0.2, tmax=0.5
epochs = mne.Epochs(raw = raw, events = events, event_id=None, tmin = period_start, tmax = period_end, preload = True)
evoked = epochs.average()
print(epochs.info)

<Info | 16 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 bads: []
 ch_names: MEG0113, MEG0112, MEG0111, MEG0122, MEG0123, MEG0121, MEG0132, ...
 chs: 204 GRAD, 102 MAG, 7 EMG, 1 ECG, 15 MISC, 3 STIM, 9 CHPI
 custom_ref_applied: False
 dev_head_t: MEG device -> head transform
 dig: 163 items (3 Cardinal, 4 HPI, 156 Extra)
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 lowpass: 330.0 Hz
 meas_date: 2016-12-28 09:46:14 UTC
 meas_id: 4 items (dict)
 nchan: 341
 proc_history: 1 item (list)
 projs: []
 sfreq: 1000.0 Hz
 subject_info: 6 items (dict)
>


In [40]:
conductivity = [0.3] # for single layer
model = mne.make_bem_model(subject=subject, ico=4, conductivity= conductivity, subjects_dir=subjects_dir, verbose=None)

bem = mne.make_bem_solution(model)

In [41]:
mne.write_bem_solution('/home/pultsinak/Рабочий стол/test_bem.h5', bem, overwrite=True, verbose=None)

In [42]:
bem = mne.read_bem_solution('/home/pultsinak/Рабочий стол/Sources/test_bem.h5', verbose=None)

FileNotFoundError: fname "/home/pultsinak/Рабочий стол/Sources/test_bem.h5" does not exist

In [43]:
mne.viz.plot_bem(subject, subjects_dir, brain_surfaces='white', orientation='coronal')

<Figure size 780x706.5 with 12 Axes>

In [44]:
fig = mne.viz.plot_bem(subject, subjects_dir, brain_surfaces='white', orientation='coronal', show = False)

In [45]:
src = mne.setup_source_space(subject=subject, add_dist=False )

In [46]:
src.plot()

In [47]:
trans = '/net/server/mnt/Archive/prob_learn/freesurfer/P001/mri/T1-neuromag/sets/P001-COR.fif'

In [48]:
mne.viz.plot_alignment(epochs.info, trans, subject=subject, surfaces=['white', 'outer_skin'])

In [49]:
# Forward Model
fwd = mne.make_forward_solution(info=epochs.info, trans=trans, src=src, bem=bem)

In [50]:
events_neg = np.loadtxt("/net/server/data/Archive/prob_learn/vtretyakova/Nikita_mio_cleaned/fix_cross_mio_corr/P001_run2_norisk_fb_cur_negative_fix_cross.txt", dtype='int')
epochs_cov = mne.Epochs(raw = raw, events = events_neg, event_id=None, tmin = -1.0, tmax = 1.0)

In [51]:
cov = mne.compute_covariance(epochs=epochs_cov, method='auto', tmin=-0.35, tmax = -0.05)

<ipython-input-51-c34ea388393d>:1: RuntimeWarning: Something went wrong in the data-driven estimation of the data rank as it exceeds the theoretical rank from the info (306 > 71). Consider setting rank to "auto" or setting it explicitly as an integer.
  cov = mne.compute_covariance(epochs=epochs_cov, method='auto', tmin=-0.35, tmax = -0.05)


In [52]:
inv = mne.minimum_norm.make_inverse_operator(raw.info, fwd, cov, loose=0.2) 


<ipython-input-52-12624a596126>:1: RuntimeWarning: Something went wrong in the data-driven estimation of the data rank as it exceeds the theoretical rank from the info (306 > 71). Consider setting rank to "auto" or setting it explicitly as an integer.
  inv = mne.minimum_norm.make_inverse_operator(raw.info, fwd, cov, loose=0.2)
<ipython-input-52-12624a596126>:1: RuntimeWarning: Something went wrong in the data-driven estimation of the data rank as it exceeds the theoretical rank from the info (306 > 71). Consider setting rank to "auto" or setting it explicitly as an integer.
  inv = mne.minimum_norm.make_inverse_operator(raw.info, fwd, cov, loose=0.2)


In [53]:
stc = mne.minimum_norm.apply_inverse_epochs(epochs, inv, lambda2=1./ 9., pick_ori='normal')

In [54]:
stc[0].plot() 

In [55]:
len(stc)


8

In [56]:
morph = mne.compute_source_morph(stc[0], subject_from=subject, subject_to='fsaverage')
stc_fsaverage0 = morph.apply(stc[0])

In [57]:
stc_fsaverage0.data.shape

(20484, 4501)

In [58]:
stc_fsaverage0

<SourceEstimate | 20484 vertices, subject : fsaverage, tmin : -1750.0 (ms), tmax : 2750.0 (ms), tstep : 1.0 (ms), data shape : (20484, 4501), ~703.6 MB>

In [59]:
snr = 3
lambda2 = 1. / snr ** 2

stc = mne.minimum_norm.apply_inverse(evoked, inv, lambda2)

In [60]:
stc.plot(hemi='both') 

In [61]:
morph = mne.compute_source_morph(stc, subject_from=subject, subject_to='fsaverage')
stc_fsaverage = morph.apply(stc)

In [62]:
stc_fsaverage.plot(hemi='both')


In [2]:

L_freq = 16
H_freq = 30
f_step = 2

#time_bandwidth = 4 #(by default = 4)
# if delta (1 - 4 Hz) 
#n_cycles = np.array([1, 1, 1, 2]) # уточнить


period_start = -1.750
period_end = 2.750

baseline = (-0.35, -0.05)

freq_range = 'beta_16_30'

description = 'STC for risk negative and positive FB'

# This code sets an environment variable called SUBJECTS_DIR
os.environ['SUBJECTS_DIR'] = '/net/server/data/Archive/prob_learn/freesurfer'
subjects_dir = '/net/server/data/Archive/prob_learn/freesurfer'

# 40 subjects with all choice types
subjects = pd.read_csv('/home/vtretyakova/Рабочий стол/probability_learning/sources/subj_list.csv')['subj_list'].tolist()

subjects.remove('P062') #without MRI
subjects.remove('P052') # bad segmentation 


rounds = [1, 2, 3, 4, 5, 6]

trial_type = ['risk'] # on this step we'll make stc only for risk positive and negative fb
#trial_type = ['norisk', 'prerisk', 'risk', 'postrisk']

feedback = ['positive', 'negative']

data_path = '/net/server/data/Archive/prob_learn/vtretyakova/ICA_cleaned'
os.makedirs('/net/server/data/Archive/prob_learn/vtretyakova/sources/{0}'.format(freq_range), exist_ok = True)
os.makedirs('/net/server/data/Archive/prob_learn/vtretyakova/sources/{0}/{0}_stc_fsaverage'.format(freq_range), exist_ok = True)

########################## Обязательно делать файл, в котором будет показано какие параметры были заданы, иначе проверить вводные никак нельзя, а это необходимо при возникновении некоторых вопросов ############################################

lines = ["freq_range = {}".format(freq_range), description, "L_freq = {}".format(L_freq), "H_freq = {}, в питоне последнее число не учитывается, т.е. по факту частота (H_freq -1) ".format(H_freq), "f_step = {}".format(f_step), "period_start = {}".format(period_start), "period_end = {}".format(period_end), "baseline = {}".format(baseline)]


with open("/net/server/data/Archive/prob_learn/vtretyakova/sources/{0}/{0}_stc_fsaverage/config.txt".format(freq_range), "w") as file:
    for  line in lines:
        file.write(line + '\n')


##############################################################################################################


for subj in subjects:
    bem = mne.read_bem_solution('/net/server/data/Archive/prob_learn/vtretyakova/sources/bem/{0}_bem.h5'.format(subj), verbose=None)
    src = mne.setup_source_space(subject =subj, add_dist=False )
    for r in rounds:
        for cond in trial_type:
            for fb in feedback:
                try:
                    
                    stc_fsaverage = make_freq_stc(subj, r, cond, fb, data_path, L_freq, H_freq, f_step, period_start, period_end, baseline, bem, src)
                    stc_fsaverage.save('/net/server/data/Archive/prob_learn/vtretyakova/sources/{0}/{0}_stc_fsaverage/{1}_run{2}_{3}_fb_cur_{4}_{0}_stc'.format(freq_range, subj, r, cond, fb))
                except (OSError):
                    print('This file not exist')

NameError: name 'os' is not defined

In [3]:
def make_freq_stc(subj, r, cond, fb, data_path, L_freq, H_freq, f_step, period_start, period_end, baseline, bem, src):
    
    bands = dict(beta=[L_freq, H_freq])
    #freqs = np.arange(L_freq, H_freq, f_step)
    
    #read events
	#events for baseline
	# download marks of positive feedback
	
    events_pos = np.loadtxt("/net/server/data/Archive/prob_learn/vtretyakova/Nikita_mio_cleaned/fix_cross_mio_corr/{0}_run{1}_norisk_fb_cur_positive_fix_cross.txt".format(subj, r), dtype='int') 
    

        # если только одна метка, т.е. одна эпоха, то выдается ошибка, поэтому приводим shape к виду (N,3)
    if events_pos.shape == (3,):
        events_pos = events_pos.reshape(1,3)
        
    # download marks of negative feedback      
    
    events_neg = np.loadtxt("/net/server/data/Archive/prob_learn/vtretyakova/Nikita_mio_cleaned/fix_cross_mio_corr/{0}_run{1}_norisk_fb_cur_negative_fix_cross.txt".format(subj, r), dtype='int')
    
    
    # если только одна метка, т.е. одна эпоха, то выдается ошибка, поэтому приводим shape к виду (N,3)
    if events_neg.shape == (3,):
        events_neg = events_neg.reshape(1,3) 
    
    #объединяем негативные и позитивные фидбеки для получения общего бейзлайна по ним, и сортируем массив, чтобы времена меток шли в порядке возрастания    
    events = np.vstack([events_pos, events_neg])
    events = np.sort(events, axis = 0) 
    
    #events, which we need
    events_response = np.loadtxt('/net/server/data/Archive/prob_learn/vtretyakova/Nikita_mio_cleaned/events_by_cond_mio_corrected/{0}_run{1}_{2}_fb_cur_{3}.txt'.format(subj, r, cond, fb), dtype='int')
    # если только одна метка, т.е. одна эпоха, то выдается ошибка, поэтому приводи shape к виду (N,3)
    if events_response.shape == (3,):
        events_response = events_response.reshape(1,3)
    
	           
    raw_fname = op.join(data_path, '{0}/run{1}_{0}_raw_ica.fif'.format(subj, r))

    raw_data = mne.io.Raw(raw_fname, preload=True)
        
    
    picks = mne.pick_types(raw_data.info, meg = True, eog = True)
		    
	# Forward Model
    trans = '/net/server/mnt/Archive/prob_learn/freesurfer/{0}/mri/T1-neuromag/sets/{0}-COR.fif'.format(subj)
        
	   	    
    #epochs for baseline
    # baseline = None, чтобы не вычитался дефолтный бейзлайн
    epochs_bl = mne.Epochs(raw_data, events, event_id = None, tmin = -1.0, tmax = 1.0, baseline = None, picks = picks, preload = True)
    cov = mne.compute_covariance(epochs=epochs_bl, method='auto', tmin=-0.35, tmax = -0.05)
     
    epochs_bl.resample(100)
    
    ####### ДЛЯ ДАННЫХ ##############
    # baseline = None, чтобы не вычитался дефолтный бейзлайн
epochs = mne.Epochs(raw_data, events_response, event_id = None, tmin = period_start, 
         tmax = period_end, baseline = None, picks = picks, preload = True)


                
fwd = mne.make_forward_solution(info=epochs.info, trans=trans, src=src, bem=bem)	                
inv = mne.minimum_norm.make_inverse_operator(raw_data.info, fwd, cov, loose=0.2) 	                
epochs.resample(100) 
    
stc_baseline = mne.minimum_norm.source_band_induced_power(epochs_bl.pick('meg'), inv, bands, use_fft=False, df = f_step, n_cycles = 8)["beta"].crop(tmin=baseline[0], tmax=baseline[1], include_tmax=True)
    
    
    #усредняем по времени
    b_line  = stc_baseline.data.mean(axis=-1)
    
    stc = mne.minimum_norm.source_band_induced_power(epochs.pick('meg'), inv, bands, use_fft=False, df = f_step, n_cycles = 8)["beta"]
    
    stc.data = 10*np.log10(stc.data/b_line[:, np.newaxis])
    
    morph = mne.compute_source_morph(stc, subject_from=subj, subject_to='fsaverage')
    stc_fsaverage = morph.apply(stc)

    return (stc_fsaverage)   